In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skvideo.io
import csv
import os
from tqdm import tqdm
import cv2 as cv
from scipy.optimize import minimize
%matplotlib inline
# np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# flow_mat = None
# image_scale = 0.5
# pyr_images = 1 # was 3
# win_size = 15
# pyr_iterations = 2 # was 3
# poly_expans = 5
# std = 1.3 # was 1.2

flow_mat = None
image_scale = 0.5
pyr_images = 1 # was 3
win_size = 10
pyr_iterations = 2 # was 3
poly_expans = 5
std = 1.3 # was 1.2

In [ ]:
cap = cv.VideoCapture(cv.samples.findFile("../train/videos/trash/trainGray_Clipped.m4v"))
# cap = cv.VideoCapture(cv.samples.findFile("../train/videos/trainGray.mp4"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
# prvs = frame1
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    if ret == True:
        next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
#         next = frame2
        flow = cv.calcOpticalFlowFarneback(prvs,next, flow_mat, image_scale, pyr_images, win_size, pyr_iterations, poly_expans, std, 0)
        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
        cv.imshow('frame2',bgr)
        k = cv.waitKey(50) & 0xff
        if k == 27:
            break
        prvs = next
    else:
        break
cap.release()
cv.destroyAllWindows()

In [ ]:
grad = ((np.dot(z, alpLmbTheta[idx, idx2]) - trainY[j])*z)/m
grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])

In [ ]:
for i in range(100):
    rand_num = np.random.uniform()
    bright_factor = 0.2 + rand_num
    print(rand_num, bright_factor)

In [ ]:
a = np.arange(18).reshape(9,2)
# b = a.reshape(3,3,2).swapaxes(0,2)
b = a.reshape(2,3,3)
print(a.shape)
print(a)
print()
print(b.shape)
print(b)
print()
b.shape = (9,2)
print(b.shape)
print(b)

In [2]:
def adjust_Value(prvs_FRAME, curr_FRAME, HSV_value):
    prvs_FRAME = cv.cvtColor(prvs_FRAME, cv.COLOR_BGR2HSV)
    curr_FRAME = cv.cvtColor(curr_FRAME, cv.COLOR_BGR2HSV)
    prvs_FRAME[...,2] = prvs_FRAME[...,2]*HSV_value
    curr_FRAME[...,2] = curr_FRAME[...,2]*HSV_value
    return prvs_FRAME, curr_FRAME

In [3]:
def opticalFlow_DENSE(prvs_FRAME, curr_FRAME):
    # flow_mat = None
    # image_scale = 0.5
    # pyr_images = 1 # was 3
    # win_size = 15
    # pyr_iterations = 2 # was 3
    # poly_expans = 5
    # std = 1.3 # was 1.2
    
    flow_mat = None
    image_scale = 0.5
    pyr_images = 1 # was 3
    win_size = 10
    pyr_iterations = 2 # was 3
    poly_expans = 5
    std = 1.3 # was 1.2

    hsv = np.zeros_like(prvs_FRAME)
    hsv[...,1] = 255
    prvs_FRAME = cv.cvtColor(prvs_FRAME,cv.COLOR_BGR2GRAY)
    curr_FRAME = cv.cvtColor(curr_FRAME,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs_FRAME,curr_FRAME,flow_mat,image_scale,pyr_images,win_size,pyr_iterations,poly_expans,std,0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    return bgr

In [4]:
def getData(train_meta, kLoopCount, learn_index):
    print('In getData')
    global img_pix_count
    opt_flows = np.empty((0,img_pix_count))
    speed = np.empty((0))
    for k in range(kLoopCount):
        HSV_value = np.random.uniform(low=.7, high=1.4)
        curr_FRAME = cv.imread(train_meta['image_path'][learn_index[k]])
        prvs_FRAME = cv.imread(train_meta['image_path'][learn_index[k]-1])
        speed = np.append(speed, train_meta['speed'][learn_index[k]])
        prvs_FRAME, curr_FRAME = adjust_Value(prvs_FRAME, curr_FRAME, HSV_value)
        opt_flow = opticalFlow_DENSE(prvs_FRAME, curr_FRAME)
        opt_flow.shape = img_pix_count
        opt_flows = np.append(opt_flows, [opt_flow], axis=0)
    print('Opt FLOWS:', opt_flows.shape, 'Type:', opt_flows.dtype)
    return opt_flows, speed

In [9]:
def linearCost(X, y, m):
    J = np.sum((np.dot(X, theta) - y)**2)/m
    return J
    
def linearDer(X, y, m):
    grad = ((np.dot(X, theta) - y)*X)/m
    grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*theta[1:len(theta)])
    return grad

In [15]:
def learn(X, y, m, loopCount):
    print('In learn')
    #z = np.array(trainX[j], np.dtype('float64'))
#     grad = ((np.dot(z, alpLmbTheta[idx, idx2]) - trainY[j])*z)/m
#     grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])
#     alpLmbTheta[idx,idx2] = (alpLmbTheta[idx,idx2] - (alp*grad))
    #res = minimize(linearCost, X, args=(y,m), method='BFGS', jac=linearDer, options={'gtol': 1e-6, 'disp': True, 'maxIter': loopCount})
    print(X.shape, y.shape)
    res = minimize(linearCost, X, args=(y,m), method='BFGS', jac=linearDer, options={'disp': True, 'maxiter': loopCount})
    #res = minimize(rosen, x0, method='BFGS', jac=rosen_der, options={'gtol': 1e-6, 'disp': True})

In [16]:
def getCost(train_meta, learn_index, train_length, jLoopCount, kLoopCount):
    print('In getCost')
    cost = 0
    index = 0
    for j in range(jLoopCount):
        kLoopCount = kLoopCount if (index+kLoopCount)<=train_length else (train_length-index)
        X, y = getData(train_meta, kLoopCount, learn_index[index:index+kLoopCount])
        print('X:', X.shape, 'y:', y.shape)
        index = index+kLoopCount
        cost = cost + linearCost(X, y, 1)
        print('cost:',cost)
    #cost = cost/learn_index.shape[0]
    cost = cost/(jLoopCount*kLoopCount)
    print('COST:', cost)

In [17]:
def startLearn(train_csv_path):
    print('In startLearn')
    train_meta = pd.read_csv(os.path.join(train_csv_path, 'trainGrayClipped_meta.csv'))
    
    train_frames = train_meta.shape[0]
    train_length = int(.8*train_frames-1)
    print('Train Length:', train_length)
    data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
    learn_index = data_index[0:train_length]
    print('Learn Index:', learn_index.shape)
    
    iLoopCount = 1
    kLoopCount = 10
    jLoopCount = 1 #learn_index/kLoopCount if learn_index%kLoopCount==0 else (int(learn_index/kLoopCount)+1)
    learnLoopCount = 10
    m = kLoopCount
    
    getCost(train_meta, learn_index, train_length, jLoopCount, kLoopCount)
    for i in range(iLoopCount):
        index = 0
        for j in range(jLoopCount):
            kLoopCount = kLoopCount if (index+kLoopCount)<=train_length else (train_length-index)
            opt_flow, speed = getData(train_meta, kLoopCount, learn_index[index:index+kLoopCount])
            index = index + kLoopCount
            learn(opt_flow, speed, m, learnLoopCount)
        getCost(train_meta, learn_index, train_length, jLoopCount, kLoopCount)

In [18]:
train_csv_path = '../train/text/CSV'

img_pix_count = (cv.imread('../train/photos/images/gray_Clipped/0.jpg')).shape
img_pix_count = (img_pix_count[0] * img_pix_count[1])*3
print('Pixel Count:', img_pix_count)

# Global Vars
theta = np.zeros(img_pix_count)

startLearn(train_csv_path)

Pixel Count: 130560
In startLearn
Train Length: 16319
Learn Index: (16319,)
In getCost
In getData
Opt FLOWS: (10, 130560) Type: float64
X: (10, 130560) y: (10,)
cost: 1506.1030517579998
COST: 150.61030517579997
In getData
Opt FLOWS: (10, 130560) Type: float64
In learn
(10, 130560) (10,)


ValueError: shapes (1305600,) and (130560,) not aligned: 1305600 (dim 0) != 130560 (dim 0)